<a href="https://colab.research.google.com/github/morenourey/CHAT/blob/main/Mentoria_Regresion_LASSO_RIDGE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Settings for the visualizations
import matplotlib.pylab as plt
%matplotlib inline
plt.rc('font', size=12)
plt.rc('figure', figsize = (12, 5))

import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("notebook", font_scale=1, rc={"lines.linewidth": 2,'font.family': [u'times']})

In [ ]:
import numpy as np
import pandas as pd

from sklearn import datasets
from sklearn.model_selection import train_test_split

from sklearn import linear_model
from sklearn.linear_model import LinearRegression


from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
seed = 42

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_boston = pd.read_csv("/content/drive/MyDrive/DS/DSC/2805/stuff/boston_house_prices.csv",header=1)
df_boston.rename(columns={"MEDV": "PRICE"}, inplace=True)

In [ ]:
boston_features = df_boston.drop(columns=['PRICE']).columns
boston_features

Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT'],
      dtype='object')

In [ ]:
df_boston.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [ ]:
X_boston = df_boston.drop(columns=['PRICE']).values
X_boston.shape

(506, 13)

In [ ]:
y_boston = df_boston.PRICE
y_boston.shape

(506,)

In [ ]:
# Train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_boston, y_boston, test_size=0.3, random_state=seed)

print('Train and test sizes of X: {} {}'.format(X_train.shape, X_test.shape))
print('Train and test sizes of y: {} {}'.format(y_train.shape, y_test.shape))

In [ ]:
# Fitting a multiple linear model
lr = LinearRegression() # Create the Linear Regression estimator
lr.fit(X_train, y_train) # Perform the fitting


# Regrerssion coefs
coefs_lr = pd.Series(np.abs(lr.coef_), boston_features).sort_values()

# Prediction
y_test_pred = lr.predict(X_test)

# evaluation
mse = mean_squared_error(y_test, y_test_pred)
r2score = lr.score(X_test, y_test)

print('\nIntercept and coefs:\n{} {}'.format(lr.intercept_, lr.coef_))
print('\nMSE: {}'.format(mse))
print('R^2 Score test: {}'.format(r2score))

In [ ]:
coefs_lr.plot(kind='bar', title='Model Coefficients')

In [ ]:
coefs_lr

Podemos ver que todos los coeficientes obtenidos son diferentes de cero, lo que significa que no se descarta ninguna variable.
A continuación, intentaremos construir un nuevo modelo para predecir el precio utilizando los factores más importantes y descartando los no informativos. Para ello, podemos crear un regresor con LASSO, forzando coeficientes cero:

## Regularization Models

### L2 Regularization: Ridge Regression.

La regresión Ridge penaliza los coeficientes si se alejan demasiado de cero, lo que obliga a que sean pequeños de forma continua. De este modo, disminuye la complejidad del modelo manteniendo todas las variables en él.

$$ minimize(\sum_{i=0}^n (y_i - \beta_0- \sum_{j=1}^p \beta_jx_{ij})^2 + \alpha\sum_{j=1}^p \beta_j^2) $$

where $\beta_j$ are the regression coefficients.


### L1 Regularization: Lasso Regression

A menudo, en los problemas reales, hay variables no informativas en los datos que impiden una modelización adecuada del problema y, por tanto, la construcción de un modelo de regresión correcto. En tales casos, es crucial un proceso de selección de características para seleccionar sólo las características informativas y descartar las no informativas. Esto puede lograrse mediante métodos dispersos que utilizan un enfoque de penalización, como *Lasso* (operador de selección y reducción mínima absoluta) para poner a cero algunos coeficientes del modelo (descartando así esas variables). La sparsity puede verse como una aplicación de la navaja de Occam: preferir modelos más simples a los complejos.

Para ello, la regresión Lasso añade un término de regularización **$\ell_1$-norma** a la suma de errores cuadrados de predicción (SSE).  Dado el conjunto de muestras (𝑋,𝐲), el objetivo es minimizar:

$$ minimize(\sum_{i=0}^n (y_i - \beta_0- \sum_{j=1}^p \beta_jx_{ij})^2 + \alpha\sum_{j=1}^p|\beta_j|)$$



### Ridge

In [ ]:
## Ridge Regression
ridge = linear_model.Ridge(alpha=1) # Create a Ridge regressor
ridge.fit(X_train, y_train) # Perform the fitting

# Regrerssion coefs
coefs_ridge = pd.Series(np.abs(ridge.coef_), boston_features).sort_values()

# Prediction
y_test_pred_ridge = ridge.predict(X_test)

# evaluation
mse_ridge = mean_squared_error(y_test, y_test_pred_ridge)
r2score_ridge = ridge.score(X_test, y_test)

print('\nIntercept and coefs:\n{} {}'.format(ridge.intercept_, ridge.coef_))
print('\nMSE: {}'.format(mse_ridge))
print('R^2 Score: {}'.format(r2score_ridge))

In [ ]:
coefs_ridge.plot(kind='bar', title='Ridge Coefficients')

### Lasso


In [ ]:
## Lasso Regression
lasso = linear_model.Lasso(alpha=1)
lasso.fit(X_train, y_train)

# Regrerssion coefs
coefs_lasso = pd.Series(np.abs(lasso.coef_), boston_features).sort_values()

# Prediction
y_test_pred_lasso = lasso.predict(X_test)

# evaluation
mse_lasso = mean_squared_error(y_test, y_test_pred_lasso)
r2score_lasso = lasso.score(X_test, y_test)


# The coefficients
print('\nIntercept and coefs:\n{} {}'.format(lasso.intercept_, lasso.coef_))

# The mean squared error
print('\nMSE: {}'.format(mse_lasso))

# The coefficient of determination: 1 is perfect prediction
print('R^2 Score: {}'.format(r2score_lasso))

In [ ]:
coefs_lasso.plot(kind='bar', title='Lasso Coefficients')

#### Comparamos resultados:

In [ ]:
f = plt.figure(figsize=(15,5))
ax1 = f.add_subplot(131)
ax2 = f.add_subplot(132)
ax3 = f.add_subplot(133)

coefs_lr.plot(kind="barh", title='coefs_lr', ax=ax1)
coefs_ridge.plot(kind="barh", title='coefs_ridge', ax=ax2)
coefs_lasso.plot(kind="barh", title='coefs_lasso', ax=ax3)

En el gráfico presentado, se muestran los coeficientes de diferentes modelos de regresión: regresión lineal (LR), regresión ridge (Ridge) y regresión lasso (Lasso). La razón por la cual los coeficientes en el eje x tienen escalas distintas se debe a las características y regularizaciones propias de cada modelo.

**Regresión Lineal (LR):**

La regresión lineal estándar no aplica ninguna regularización. Esto significa que los coeficientes pueden tomar cualquier valor que minimice el error cuadrático medio (MSE) sin restricciones adicionales. Como resultado, los coeficientes pueden ser relativamente grandes si los datos lo permiten, como se observa en el gráfico donde el coeficiente para NOX es muy alto comparado con los demás.

**Regresión Ridge:**

La regresión ridge incluye una penalización L2 que agrega la suma de los cuadrados de los coeficientes al objetivo de minimización. Esta penalización reduce los valores de los coeficientes pero no los lleva exactamente a cero. La penalización controla el tamaño de los coeficientes, resultando en coeficientes más pequeños que en la regresión lineal estándar, pero aún presentes. Esto se observa en el gráfico de coef_ridge donde los coeficientes están en una escala más pequeña en comparación con los de la regresión lineal.


**Regresión Lasso:**

La regresión lasso incluye una penalización L1 que agrega la suma de los valores absolutos de los coeficientes al objetivo de minimización. Esta penalización no solo reduce los valores de los coeficientes, sino que también puede llevar algunos coeficientes exactamente a cero, efectivamente seleccionando un subconjunto de características. Debido a esta propiedad de seleccionar características, los coeficientes tienden a ser aún más pequeños o nulos, como se observa en el gráfico coef_lasso.


https://es.images.search.yahoo.com/search/images;_ylt=Awr.oNuHZ05mmuEI6CJU04lQ;_ylu=Y29sbwNpcjIEcG9zAzEEdnRpZAMEc2VjA3BpdnM-?p=regularization+ridge+lasso&fr2=piv-web&type=E210ES91213G0&fr=mcafee#id=27&iurl=https%3A%2F%2Ffavtutor.com%2Fresources%2Fimages%2Fuploads%2Fmceu_95973090761620302179448.png&action=click

**Contornos de la Función de Costo:**

Las elipses rojas representan los contornos de la función de costo de la regresión lineal.
La intersección de estas elipses con la región permitida por la restricción determina los valores óptimos de los coeficientes (
𝛽
β) en presencia de regularización.


***Lasso:***

La forma del rombo tiene vértices en los ejes, lo que significa que es más probable que se produzca una intersección en uno de los ejes, resultando en coeficientes exactamente cero.

***Ridge:***

La forma circular significa que la intersección es más probable en algún lugar dentro del círculo, lo que generalmente produce coeficientes diferentes de cero pero pequeños.


**Conclusión**

Lasso es útil cuando buscamos un modelo más interpretable que seleccione automáticamente un subconjunto de características.

Ridge es útil cuando todas las características son relevantes y preferimos no descartar ninguna, pero deseamos reducir sus magnitudes para evitar sobreajuste.
Ambos métodos ayudan a mejorar el rendimiento del modelo y la generalización al evitar el sobreajuste, pero lo hacen de maneras ligeramente diferentes debido a sus diferentes tipos de restricciones.

In [ ]:
coefs_ridge

In [ ]:
coefs_lasso

In [ ]:
scores = [[r2score, mse],
         [r2score_ridge, mse_ridge],
         [r2score_lasso, mse_lasso]]
df_scores = pd.DataFrame(scores, columns=["R2", "mse"], index=["No regularization", "Ridge", "Lasso"])
df_scores

## Transform and Predict

In [ ]:
for i, feat in enumerate(boston_features):
    print()
    print(feat)
    print("Max {}, min {}, mean {}, and var {}".format(np.max(X_boston[:, i]), np.min(X_boston[:, i]), np.mean(X_boston[:, i]), np.var(X_boston[:, i])))

Hay un tipo especial de ``Estimator`` llamado ``Transformer`` que transforma los datos de entrada -- por ejemplo, selecciona un subconjunto de las características o extrae nuevas características basadas en las originales.

Un transformador que utilizaremos aquí es ``sklearn.preprocessing.StandardScaler``. Este transformador centra cada predictor en ``X`` para tener media cero y varianza unitaria

In [ ]:
from sklearn.preprocessing import StandardScaler

scalerX = StandardScaler().fit(X_train) # Create the transformer StandardScaler and perform the fitting for the training data

X_train_norm = scalerX.transform(X_train)
X_test_norm = scalerX.transform(X_test)

print("\nBefore transformation:")
print('Train: Max {}, min {}, mean {}, and var {}'.format(np.max(X_train), np.min(X_train), np.mean(X_train), np.var(X_train)))
print('Test: Max {}, min {}, mean {}, and var {}'.format(np.max(X_test), np.min(X_test), np.mean(X_test), np.var(X_test)))

print("\nAfter transformation:")
print('Train: Max {}, min {}, mean {}, and var {}'.format(np.max(X_train_norm), np.min(X_train_norm), np.mean(X_train_norm), np.var(X_train_norm)))
print('Test: Max {}, min {}, mean {}, and var {}'.format(np.max(X_test_norm), np.min(X_test_norm), np.mean(X_test_norm), np.var(X_test_norm)))

In [ ]:
# Train model
lr_norm = linear_model.LinearRegression()
ridge_norm = linear_model.Ridge(alpha=.3)
lasso_norm = linear_model.Lasso(alpha=.3)

lr_norm.fit(X_train_norm, y_train)
ridge_norm.fit(X_train_norm, y_train)
lasso_norm.fit(X_train_norm, y_train)

coefs_lr_norm = pd.Series(np.abs(lr_norm.coef_), boston_features).sort_values()
coefs_ridge_norm = pd.Series(np.abs(ridge_norm.coef_), boston_features).sort_values()
coefs_lasso_norm = pd.Series(np.abs(lasso_norm.coef_), boston_features).sort_values()

In [ ]:
# Before normalization
f = plt.figure(figsize=(15,5))
ax1 = f.add_subplot(131)
ax2 = f.add_subplot(132)
ax3 = f.add_subplot(133)

coefs_lr.plot(kind="barh", title='coefs_lr', ax=ax1)
coefs_ridge.plot(kind="barh", title='coefs_ridge', ax=ax2)
coefs_lasso.plot(kind="barh", title='coefs_lasso', ax=ax3)

In [ ]:
f = plt.figure(figsize=(15,5))
ax1 = f.add_subplot(131)
ax2 = f.add_subplot(132)
ax3 = f.add_subplot(133)

coefs_lr_norm.plot(kind="barh", title='coefs_lr_norm', ax=ax1)
coefs_ridge_norm.plot(kind="barh", title='coefs_ridge_norm', ax=ax2)
coefs_lasso_norm.plot(kind="barh", title='coefs_lasso_norm', ax=ax3)

In [ ]:
# Scores
print('lr: {}'.format(lr.score(X_test, y_test)))
print('ridge: {}'.format(ridge.score(X_test, y_test)))
print('lasso: {}'.format(lasso.score(X_test, y_test)))
print('lr_norm: {}'.format(lr_norm.score(X_test_norm, y_test)))
print('ridge_norm: {}'.format(ridge_norm.score(X_test_norm, y_test)))
print('lasso_norm: {}'.format(lasso_norm.score(X_test_norm, y_test)))

#### Interpretación de Resultados

1. **Multicolinealidad Moderada**
   - La multicolinealidad en el Boston dataset no es lo suficientemente problemática para justificar el uso de Ridge. Esto significa que las características no están tan altamente correlacionadas entre sí como para que la regularización L2 de Ridge ofrezca una mejora significativa en el rendimiento del modelo.

2. **Relevancia de las Características**
   - Todas las características del Boston dataset parecen ser relevantes para predecir la variable objetivo. La penalización L1 de Lasso está eliminando algunas de estas características importantes, lo que lleva a un peor rendimiento en comparación con la regresión lineal sin regularización.

3. **Efecto de la Normalización**
   - La normalización de los datos no cambia significativamente los resultados. Esto se debe a que la regresión lineal sin regularización ya está capturando bien las relaciones lineales en los datos. Aunque la normalización mejora ligeramente el rendimiento de Ridge y Lasso, no es suficiente para superar a la regresión lineal simple.

#### Conclusión

En resumen, la regresión lineal sin regularización está funcionando mejor con el Boston dataset porque:
- La multicolinealidad no es lo suficientemente problemática como para requerir Ridge.
- Todas las características son relevantes, por lo que Lasso está eliminando características importantes y deteriorando el rendimiento.
- La normalización no tiene un impacto significativo porque la regresión lineal sin regularización ya captura bien las relaciones en los datos.

### Tuning Ridge and Lasso regressors

In [ ]:
n_alphas = 100
alphas = np.logspace(-2, 2, n_alphas)

coefs_ridge = []
r2_ridge = []
for l in alphas:
    regr_ridge = linear_model.Ridge(alpha=l) # Create a Ridge regressor
    regr_ridge.fit(X_train_norm, y_train)  # Perform the fitting
    coefs_ridge.append(regr_ridge.coef_)
    r2_ridge.append(regr_ridge.score(X_test_norm,y_test))


coefs_lasso = []
r2_lasso = []
for l in alphas:
    regr_lasso = linear_model.Lasso(alpha=l,tol =0.001) # Create a Ridge regressor
    regr_lasso.fit(X_train_norm, y_train)  # Perform the fitting
    coefs_lasso.append(regr_lasso.coef_)
    r2_lasso.append(regr_lasso.score(X_test_norm,y_test))

α: Es el parámetro de regularización que controla la magnitud de la penalización en ambos modelos. Un
𝛼
α mayor implica una mayor regularización, lo que tiende a simplificar el modelo y reducir el overfitting.

In [ ]:
# Display results

fig, axs = plt.subplots(2, 2, figsize=(20, 20), sharey='row')


axs[0,0].plot(alphas, np.abs(coefs_ridge))
axs[0,0].set_xscale('log')
axs[0,0].set_title('Ridge coefficients as a function of the regularization')
axs[0,0].axis('tight')
axs[0,0].set_xlabel('alpha')
axs[0,0].set_ylabel('weights')
axs[0,0].legend(boston_features)

axs[0,1].plot(alphas, np.abs(coefs_lasso))
axs[0,1].set_xscale('log')
axs[0,1].set_title('Lasso coefficients as a function of the regularization')
axs[0,1].axis('tight')
axs[0,1].set_xlabel('alpha')
axs[0,1].set_ylabel('weights')
axs[0,1].legend(boston_features)

axs[1,0].plot(alphas, r2_ridge)
axs[1,0].set_xscale('log')
axs[1,0].set_title('Ridge scores as a function of the regularization')
axs[1,0].axis('tight')
axs[1,0].set_xlabel('alpha')
axs[1,0].set_ylabel('r2')

axs[1,1].plot(alphas, r2_lasso)
axs[1,1].set_xscale('log')
axs[1,1].set_title('Lasso scores as a function of the regularization')
axs[1,1].axis('tight')
axs[1,1].set_xlabel('alpha')
axs[1,1].set_ylabel('r2')


plt.show()

In [ ]:
# Find optimal alphas
best_r2_ridge = max(r2_ridge)
max_index_ridge = r2_ridge.index(best_r2_ridge)
best_alpha_ridge = alphas[max_index_ridge]
print('Ridge')
print(max_index_ridge, best_alpha_ridge, best_r2_ridge, r2_ridge[max_index_ridge])

best_r2_lasso = max(r2_lasso)
max_index_lasso = r2_lasso.index(best_r2_lasso)
best_alpha_lasso = alphas[max_index_lasso]
print('Lasso')
print(max_index_lasso, best_alpha_lasso, best_r2_lasso, r2_lasso[max_index_lasso])

Esto quiere decir que, para ambos modelos, el mejor rendimiento se obtiene con un alpha de 0.01. Los modelos alcanzaron un puntaje
𝑅
2
R
2
  de aproximadamente 0.7397 para Ridge y 0.7397 para Lasso, lo que sugiere que los dos modelos tienen un rendimiento muy similar con ese valor de alpha.

El índice 0 indica que este valor de alpha fue el primer elemento en la lista de alphas que probaste. En otras palabras, ambos modelos dieron su mejor rendimiento con el primer valor de alpha que se evaluó (0.01).